In [2]:
!pip install google-play-scraper

In [4]:
!pip install pymongo

In [1]:
!pip install tzlocal

In [1]:
import pandas as pd

from google_play_scraper import app, Sort, reviews

from pprint import pprint

import pymongo
from pymongo import MongoClient

import datetime as dt
from tzlocal import get_localzone

import random
import time

import os

In [2]:
client = MongoClient(host='localhost', port=27017)

app_proj_db = client['app_proj_db']

info_collection = app_proj_db['info_collection']

review_collection = app_proj_db['review_collection']

In [9]:
# app_df = pd.read_csv('Var/PlayStore/ScrapApp.csv.csv')
# app_df.head()
# INI DIGANTI APP NAME SAMA APPID
app_name = ['Facebook']
android_appID = ['com.facebook.katana']

In [10]:
app_names = list(app_name)
app_ids = list(android_appID)

In [11]:
app_info = []
for i in app_ids:
    info = app(i)
    del info['comments']
    app_info.append(info)

pprint(app_info[0])

{'adSupported': True,
 'appId': 'com.facebook.katana',
 'categories': [{'id': 'SHOPPING', 'name': 'Shopping'}],
 'containsAds': True,
 'contentRating': 'Teen',
 'contentRatingDescription': None,
 'currency': 'USD',
 'description': 'Keeping up with friends is faster and easier than ever. Share '
                'updates and photos, engage with friends and Pages, and stay '
                'connected to communities important to you. \r\n'
                '\r\n'
                'Features on the Facebook app include:\r\n'
                '\r\n'
                '* Connect with friends and family and meet new people on your '
                'social media network\r\n'
                '* Set status updates & use Facebook emoji to help relay '
                'what’s going on in your world\r\n'
                '* Share photos, videos, and your favorite memories.\r\n'
                '* Get notifications when friends like and comment on your '
                'posts\r\n'
                '* Find

In [12]:
info_collection.insert_many(app_info)

In [13]:
info_df = pd.DataFrame(list(info_collection.find({})))
info_df.head()

,_id,title,description,descriptionHTML,summary,installs,minInstalls,realInstalls,score,ratings,...,videoImage,contentRating,contentRatingDescription,adSupported,containsAds,released,updated,version,appId,url
0,64789280d6a5a2690daf08b9,Facebook,Keeping up with friends is faster and easier t...,Keeping up with friends is faster and easier t...,"Find friends, watch live videos, play games & ...","5,000,000,000+",5000000000,8682717668,3.159434,133890374,...,None,Teen,None,True,True,None,1685564158,None,com.facebook.katana,https://play.google.com/store/apps/details?id=...
1,6479264ab89d6a3c14881591,Tokopedia,"Tokopedia, Your Go-To All-in-One App!\r\n\r\nT...","Tokopedia, Your Go-To All-in-One App!<br><br>T...",Shop Anything You Want!,"100,000,000+",100000000,125954106,4.555042,6600297,...,None,Everyone,None,False,False,"Jul 26, 2015",1685066192,3.219,com.tokopedia.tkpd,https://play.google.com/store/apps/details?id=...
2,648039813430ac4b4ccf9473,Tokopedia,"Tokopedia, Your Go-To All-in-One App!\r\n\r\nT...","Tokopedia, Your Go-To All-in-One App!<br><br>T...",Shop Anything You Want!,"100,000,000+",100000000,126089710,4.556787,6605507,...,None,Everyone,None,False,False,"Jul 26, 2015",1685927369,3.220,com.tokopedia.tkpd,https://play.google.com/store/apps/details?id=...
3,64803d613430ac4b4cd030b4,Facebook,Keeping up with friends is faster and easier t...,Keeping up with friends is faster and easier t...,"Find friends, watch live videos, play games & ...","5,000,000,000+",5000000000,8697506966,3.158292,133931107,...,None,Teen,None,True,True,None,1685564158,None,com.facebook.katana,https://play.google.com/store/apps/details?id=...


In [ ]:
for app_name, app_id in zip(app_names, app_ids):
    
    start = dt.datetime.now(tz=get_localzone())
    fmt= "%m/%d/%y - %T %p"    
    
    print('---'*20)
    print('---'*20)    
    print(f'***** {app_name} started at {start.strftime(fmt)}')
    print()
    
    app_reviews = []
    
    count = 200
    
    batch_num = 0
    
    # Retrieve reviews (and continuation_token) with reviews function
    rvws, token = reviews(
        app_id,           # found in app's url
        lang='id',        # defaults to 'en'
        country='id',     # defaults to 'us'
        sort=Sort.NEWEST, # start with most recent
        count=count       # batch size
    )  # type: ignore
    
    for r in rvws:
        r['app_name'] = app_name # add key for app's name
        r['app_id'] = app_id     # add key for app's id
    
    
    app_reviews.extend(rvws)
    
    batch_num +=1 
    print(f'Batch {batch_num} completed.')
    
    time.sleep(random.randint(1,5))
    
    pre_review_ids = []
    for rvw in app_reviews:
        pre_review_ids.append(rvw['reviewId'])
    
    for batch in range(4999):
        rvws, token = reviews( # store continuation_token
            app_id,
            lang='id',
            country='id',
            sort=Sort.NEWEST,
            count=count,
            # using token obtained from previous batch
            continuation_token=token
        )
        
        new_review_ids = []
        for r in rvws:
            new_review_ids.append(r['reviewId'])
            
            r['app_name'] = app_name # add key for app's name
            r['app_id'] = app_id     # add key for app's id
    
        app_reviews.extend(rvws)
        
        batch_num +=1
        
        # Break loop and stop scraping for current app if most recent batch
        # did not add any unique reviews
        all_review_ids = pre_review_ids + new_review_ids
        if len(set(pre_review_ids)) == len(set(all_review_ids)):
            print(f'No reviews left to scrape. Completed {batch_num} batches.\n')
            break
        
        # all_review_ids becomes pre_review_ids to check against 
        # for next batch
        pre_review_ids = all_review_ids
        
        # At every 100th batch
        if batch_num%100==0:
            
            # print update on number of batches
            print(f'Batch {batch_num} completed.')
            
            # insert reviews into collection
            review_collection.insert_many(app_reviews)
            
            output_path = './' + app_name + '.csv' 
            review_df = pd.DataFrame(app_reviews)
            review_df.to_csv(output_path, mode='a', header=not os.path.exists(output_path))
            
            # print update about num reviews inserted
            store_time = dt.datetime.now(tz=get_localzone())
            print(f"""
            Successfully inserted {len(app_reviews)} {app_name} 
            reviews into collection at {store_time.strftime(fmt)}.\n
            """)
            
            # empty our list for next round of 100 batches
            app_reviews = []
        
        time.sleep(random.randint(1,5))
    
    # Print update when max number of batches has been reached
    # OR when last batch didn't add any unique reviews
    print(f'Done scraping {app_name}.')
    print(f'Scraped a total of {len(set(pre_review_ids))} unique reviews.\n')
    
    
    # Insert remaining reviews into collection
    review_collection.insert_many(app_reviews)
    
    output_path = './' + app_name + '.csv' 
    review_df = pd.DataFrame(app_reviews)
    review_df.to_csv(output_path, mode='a', header=not os.path.exists(output_path))
    
    # Get end time
    end = dt.datetime.now(tz=get_localzone())
    
    # Print ending output for app
    print(f"""
    Successfully inserted all {app_name} reviews into collection
    at {end.strftime(fmt)}.\n
    """)
    print(f'Time elapsed for {app_name}: {end-start}')
    print('---'*20)
    print('---'*20)
    print('\n')
    
    time.sleep(random.randint(1,5))

------------------------------------------------------------
------------------------------------------------------------
***** Facebook started at 06/07/23 - 15:18:46 PM

Batch 1 completed.
Batch 100 completed.

            Successfully inserted 20000 Facebook 
            reviews into collection at 06/07/23 - 15:25:58 PM.

            
Batch 200 completed.

            Successfully inserted 20000 Facebook 
            reviews into collection at 06/07/23 - 15:34:13 PM.

            
